# Векторные хранилища и ретриверы

В этом разделе вы узнаете ознакомитесь с абстракциями векторных хранилищ и ретриверов в GigaChain.
Эти абстракции предназначены для поддержки извлечения данных из векторных баз данных и других источников.
Ретриверы и векторные хранилища важны для приложений, которые извлекают данные для рассуждений в рамках инференса модели, например, в случае генерации с дополнением извлечения данных или RAG (подробнее — в [разделе про RAG](/docs/tutorials/rag)).

## Основные понятия

В этом разделе основное внимание уделяется извлечению текстовых данных. Здесь рассмотрены такие понятия, как:

- Документы;
- Векторные хранилища;
- Ретриверы.

## Подготовка к разработке

### Jupyter-блокноты

Это руководство, как и большинство других в документации, использует [Jupyter-блокноты](https://jupyter.org/). Они отлично подходят для изучения работы с LLM-системами, так как предоставляют интерактивную среду для работы с руководствами и позволяют работать с непредвиденными ситуациями: недоступностью API, нетипичным выводом и другими.

Подробнее об установке jupyter — в [официальной документации](https://jupyter.org/install).

### Установка

Для установки GigaChain выполните команды:

```{=mdx}
import Tabs from '@theme/Tabs';
import TabItem from '@theme/TabItem';
import CodeBlock from "@theme/CodeBlock";

<Tabs>
  <TabItem value="pip" label="Pip" default>
    <CodeBlock language="bash">pip install langchain</CodeBlock>
  </TabItem>
  <TabItem value="conda" label="Conda">
    <CodeBlock language="bash">conda install langchain -c conda-forge</CodeBlock>
  </TabItem>
</Tabs>
```


Подробнее об установке — в разделе [Установка](https://developers.sber.ru/docs/ru/gigachain/get-started/installation).

<!--
### LangSmith

Многие приложения, которые вы создаете с помощью GigaChain, будут содержать несколько шагов с многократными вызовами LLM.
По мере усложнения этих приложений становится важно иметь возможность инспектировать, что именно происходит внутри вашей цепочки или агента.
Лучший способ сделать это — с помощью [LangSmith](https://smith.langchain.com).

После регистрации по ссылке выше, убедитесь, что вы установили переменные среды для начала ведения журнала трассировок:

```shell
export LANGCHAIN_TRACING_V2="true"
export LANGCHAIN_API_KEY="..."
```

Или, если вы работаете в блокноте, вы можете установить их с помощью:

```python
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
```
-->


## Документы

GigaChain реализует класс [Document](https://api.python.langchain.com/en/latest/documents/langchain_core.documents.base.Document.html), которая представляет единицу текста и связанных с ним метаданных.
У этого класса есть два атрибута:

- `page_content` — строка, представляющая содержимое;
- `metadata` — словарь, содержащий произвольные метаданные.

Атрибут `metadata` может содержать данные об источнике документа, его связи с другими документами и другую информацию.
Отдельный объект `Document` часто представляет фрагмент более крупного документа.

Создайте несколько примеров документов:

In [1]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

Пример выше содержит пять документов, которые содержат метаданные, указывающие на три различных источника (`source`).

## Векторные хранилища

Векторный поиск — это распространенный способ хранения и поиска по неструктурированным данным (например, неструктурированному тексту).
Основной принцип заключается в хранении числовых векторов, которые ассоциированы с текстом.
При запросе к хранилищу, вы можете [представить текст запроса в виде вектора](/docs/concepts#embedding-models) той же размерности и использовать метрики векторной схожести для поиска подходящих данных в хранилище.

Объекты [`VectorStore`](https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStore.html) предоставляют методы для добавления текста и объектов `Document` в хранилище, а так же методы для поиска документов в хранилище с использованием различных метрик схожести.
Как правило, хранилища инициализируются с использованием моделей, которые поддерживают [векторное представление текста](/docs/how_to/embed_text).
Такие модели определяют, как текстовые данные переводятся в числовые векторы.

GigaChain поддерживает набор [интеграций](/docs/integrations/vectorstores) с различными векторными хранилищам.
Некоторые из них предоставляются на стороне других компаний (например, у различных облачных провайдеров) и требуют соответствующих учетных данных для использования.
Некоторые (такие как [Postgres](/docs/integrations/vectorstores/pgvector)) работают в отдельной инфраструктуре, которую можно запускать локально или через сторонние сервисы.
Какие-то могут работать в памяти и используются для легких нагрузок.
В этом разделе показана работа объектов `VectorStore` с использованием [Chroma](/docs/integrations/vectorstores/chroma): векторным хранилищем, которое поддерживает работу в памяти.

Как правило, для создания векторного хранилища нужно предоставить модель, которая будет отвечать за [векторное представление текста](/docs/how_to/embed_text).
В примере ниже показана работа модели [OpenAI](/docs/integrations/text_embedding/openai/).

In [2]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(
    documents,
    embedding=OpenAIEmbeddings(),
)

Вызов метода `.from_documents` добавляет документы в векторное хранилище.
Объект [`VectorStore`](https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStore.html) предоставляет методы для добавления документов, которые также можно вызвать после создания объекта.
Большинство реализаций позволят вам подключиться к существующему векторному хранилищу, например, предоставив клиент, название индекса или другую информацию.
Подробнее — в документации по [интеграции с выбранным векторным хранилищем](/docs/integrations/vectorstores).

После создания `VectorStore`, содержащего документы, мы можем выполнять его запросы. Объект [VectorStore](https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStore.html) включает методы для выполнения запросов:
- Синхронно и асинхронно;
- По строковому запросу и по вектору;
- С возвращением и без возвращения оценок схожести;
- По схожести и [максимальной маржинальной релевантности](https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStore.html#langchain_core.vectorstores.VectorStore.max_marginal_relevance_search) (чтобы сбалансировать схожесть с разнообразием в извлеченных результатах).

Методы обычно включают список объектов [Document](https://api.python.langchain.com/en/latest/documents/langchain_core.documents.base.Document.html#langchain_core.documents.base.Document) в своих выводах.

### Примеры

Возврат документов на основе схожести с текстовым запросом:

In [3]:
vectorstore.similarity_search("cat")

[Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Rabbits are social animals that need plenty of space to hop around.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Parrots are intelligent birds capable of mimicking human speech.', metadata={'source': 'bird-pets-doc'})]

Асинхронный запрос:

In [4]:
await vectorstore.asimilarity_search("cat")

[Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Rabbits are social animals that need plenty of space to hop around.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Parrots are intelligent birds capable of mimicking human speech.', metadata={'source': 'bird-pets-doc'})]

Возврат оценок схожести:

In [5]:
# Обратите внимание, что оценка зависит от выбранного векторного хранилища.
# Здесь Chroma возвращает метрику расстояния, которая должна варьироваться обратно пропорционально схожести.

vectorstore.similarity_search_with_score("cat")

[(Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
  0.3751849830150604),
 (Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'}),
  0.48316916823387146),
 (Document(page_content='Rabbits are social animals that need plenty of space to hop around.', metadata={'source': 'mammal-pets-doc'}),
  0.49601367115974426),
 (Document(page_content='Parrots are intelligent birds capable of mimicking human speech.', metadata={'source': 'bird-pets-doc'}),
  0.4972994923591614)]

Возврат документов на основе схожести с запросом, представленным в виде вектора:

In [6]:
embedding = OpenAIEmbeddings().embed_query("cat")

vectorstore.similarity_search_by_vector(embedding)

[Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Rabbits are social animals that need plenty of space to hop around.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Parrots are intelligent birds capable of mimicking human speech.', metadata={'source': 'bird-pets-doc'})]

Больше информации о векторных хранилищах:

- [Справка API](https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStore.html)
- [Работа с векторными хранилищами](/docs/how_to/vectorstores)
- [Документация для интеграции с выбранным хранилищем](/docs/integrations/vectorstores)

## Ретриверы

Объекты `VectorStore` не являются подклассами [Runnable](https://api.python.langchain.com/en/latest/core_api_reference.html#module-langchain_core.runnables) и поэтому не могут быть напрямую интегрированы в [цепочки LangChain Expression Language](/docs/concepts/#langchain-expression-language-lcel).

В то же время ретриверы GigaChain ([`Retrievers`](https://api.python.langchain.com/en/latest/core_api_reference.html#module-langchain_core.retrievers)) являются runnable-интерфейсами, поэтому они реализуют стандартный набор методов (например, синхронные и асинхронные операции `invoke` и `batch`) и предназначены для включения в цепочки LCEL.

Вы можете самостоятельно создать ретривер, не прибегая к классу `Retriever`.
Для этого нужно выбрать метод, который будет использоваться для извлечения документов, и создать runnable.
Пример ниже показывает как создать ретривер, который использует метод `similarity_search`, на основе Runnable:

In [7]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)  # select top result

retriever.batch(["cat", "shark"])

[[Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'})],
 [Document(page_content='Goldfish are popular pets for beginners, requiring relatively simple care.', metadata={'source': 'fish-pets-doc'})]]

Векторные хранилища предоставляют метод `as_retriever`, который создаст экземпляр класса Retriever, а именно [VectorStoreRetriever](https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStoreRetriever.html).
Эти ретриверы включают заданные атрибуты `search_type` и `search_kwargs`, которые определяют, какие методы базового векторного хранилища вызывать и как задавать их параметры.
Так, вы можете повторить функциональность прошлого примера с помощью следующего кода:

In [8]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(["cat", "shark"])

[[Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'})],
 [Document(page_content='Goldfish are popular pets for beginners, requiring relatively simple care.', metadata={'source': 'fish-pets-doc'})]]

`VectorStoreRetriever` поддерживает типы поиска `"similarity"` (по умолчанию), `"mmr"` (maximum marginal relevance, описано выше) и `"similarity_score_threshold"`.
Последний тип можно использовать для отсечки документов, выводимых ретривером, на основе оценки схожести.

Ретриверы можно легко включить в более сложные RAG-приложения, которые объединяют заданный вопрос с извлеченным контекстом в промпт для LLM.
Ниже показан базовый пример.

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />
```

In [9]:
# | output: false
# | echo: false

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

In [11]:
response = rag_chain.invoke("tell me about cats")

print(response.content)

Cats are independent pets that often enjoy their own space.


## Смотрите также

Стратегии извлечения данных могут быть разнообразными и сложными. Например:

- [Выведение жестких правила и фильтры](/docs/how_to/self_query/) из запроса. Например, "использовать документы, опубликованные после 2020 года";
- [Возвращение документов, каким-то образом связанных с извлеченным контекстом](/docs/how_to/parent_document_retriever/). Например, через таксономию документов;
- [Создание нескольких векторных представлений](/docs/how_to/multi_vector) для каждой единицы контекста;
- [Объединение результаты](/docs/how_to/ensemble_retriever) от нескольких ретриверов;
- [Управление весами документов](/docs/how_to/time_weighted_vectorstore/). Например, придание большего веса недавним документам.

Раздел [Работа с ретриверами](/docs/how_to#retrievers) в руководствах по использованию охватывает эти и другие стратегии извлечения данных.

Вы также можете расширить класс [BaseRetriever](https://api.python.langchain.com/en/latest/retrievers/langchain_core.retrievers.BaseRetriever.html) для реализации [собственных ретриверов](/docs/how_to/custom_retriever).